In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import scipy.stats as st

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns

In [4]:
df = pd.read_csv('data.csv', sep = ',')
df.head()

,name,gp,min,pts,fgm,fga,fg,3p_made,3pa,3p,...,fta,ft,oreb,dreb,reb,ast,stl,blk,tov,target
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,...,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,...,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,...,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,...,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,...,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0


In [5]:
df.dtypes

name        object
gp           int64
min        float64
pts        float64
fgm        float64
fga        float64
fg         float64
3p_made    float64
3pa        float64
3p         float64
ftm        float64
fta        float64
ft         float64
oreb       float64
dreb       float64
reb        float64
ast        float64
stl        float64
blk        float64
tov        float64
target     float64
dtype: object

In [6]:
df.isnull().sum()

name        0
gp          0
min         0
pts         0
fgm         0
fga         0
fg          0
3p_made     0
3pa         0
3p         11
ftm         0
fta         0
ft          0
oreb        0
dreb        0
reb         0
ast         0
stl         0
blk         0
tov         0
target      0
dtype: int64

In [7]:
df[df['3p'].isnull()]

,name,gp,min,pts,fgm,fga,fg,3p_made,3pa,3p,...,fta,ft,oreb,dreb,reb,ast,stl,blk,tov,target
338,Ken Johnson,64,12.7,4.1,1.8,3.3,52.8,0.0,0.0,NaN,...,1.3,43.5,1.4,2.4,3.8,0.3,0.2,0.3,0.9,0.0
339,Ken Johnson,64,12.7,4.1,1.8,3.3,52.8,0.0,0.0,NaN,...,1.3,43.5,1.4,2.4,3.8,0.3,0.2,0.3,0.9,0.0
340,Pete Williams,53,10.8,2.8,1.3,2.1,60.4,0.0,0.0,NaN,...,0.8,42.5,0.9,1.9,2.8,0.3,0.4,0.4,0.4,0.0
358,Melvin Turpin,79,24.7,10.6,4.6,9.0,51.1,0.0,0.0,NaN,...,1.8,78.4,2.0,3.8,5.7,0.5,0.5,1.1,1.5,1.0
386,Jim Petersen,60,11.9,3.2,1.2,2.4,48.6,0.0,0.0,NaN,...,1.1,75.8,0.7,1.7,2.5,0.5,0.2,0.5,1.2,1.0
397,Tom Scheffler,39,6.9,1.3,0.5,1.3,41.2,0.0,0.0,NaN,...,0.5,50.0,0.5,1.5,1.9,0.3,0.2,0.3,0.4,0.0
507,Sam Williams,59,18.2,6.1,2.6,4.7,55.6,0.0,0.0,NaN,...,1.5,55.1,1.5,3.7,5.2,0.6,0.8,1.3,1.1,0.0
509,Kurt Nimphius,63,17.2,5.3,2.2,4.7,46.1,0.0,0.0,NaN,...,1.7,58.3,1.5,3.2,4.7,1.0,0.3,1.3,0.9,1.0
510,Pete Verhoeven,71,17.0,4.9,2.1,4.2,50.3,0.0,0.0,NaN,...,1.0,70.8,1.5,2.1,3.6,0.7,0.6,0.3,0.8,1.0
521,Jim Smith,72,11.9,2.9,1.2,2.3,50.9,0.0,0.0,NaN,...,1.2,45.9,1.0,1.5,2.5,0.6,0.3,0.7,0.7,0.0


In [8]:
df = df.fillna(0)
df.loc[510,'3p']

0.0

In [9]:
dic = dict(zip(df.name.unique(),range(0,len(df.name.unique()))))

In [10]:
df.name = df.name.replace(dic)

In [11]:
df.head()

,name,gp,min,pts,fgm,fga,fg,3p_made,3pa,3p,...,fta,ft,oreb,dreb,reb,ast,stl,blk,tov,target
0,0,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,...,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,1,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,...,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,2,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,...,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,3,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,...,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,4,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,...,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0


In [12]:
df.target.unique()


array([0., 1.])

In [13]:
df.describe()

,name,gp,min,pts,fgm,fga,fg,3p_made,3pa,3p,...,fta,ft,oreb,dreb,reb,ast,stl,blk,tov,target
count,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,...,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000
mean,637.994776,60.414179,17.624627,6.801493,2.629104,5.885299,44.169403,0.247612,0.779179,19.149627,...,1.821940,70.300299,1.009403,2.025746,3.034478,1.550522,0.618507,0.368582,1.193582,0.620149
std,375.160274,17.433992,8.307964,4.357545,1.683555,3.593488,6.137679,0.383688,1.061847,16.051861,...,1.322984,10.578479,0.777119,1.360008,2.057774,1.471169,0.409759,0.429049,0.722541,0.485531
min,0.000000,11.000000,3.100000,0.700000,0.300000,0.800000,23.800000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.200000,0.300000,0.000000,0.000000,0.000000,0.100000,0.000000
25%,310.750000,47.000000,10.875000,3.700000,1.400000,3.300000,40.200000,0.000000,0.000000,0.000000,...,0.900000,64.700000,0.400000,1.000000,1.500000,0.600000,0.300000,0.100000,0.700000,0.000000
50%,634.500000,63.000000,16.100000,5.550000,2.100000,4.800000,44.100000,0.100000,0.300000,22.200000,...,1.500000,71.250000,0.800000,1.700000,2.500000,1.100000,0.500000,0.200000,1.000000,1.000000
75%,962.250000,77.000000,22.900000,8.800000,3.400000,7.500000,47.900000,0.400000,1.200000,32.500000,...,2.300000,77.600000,1.400000,2.600000,4.000000,2.000000,0.800000,0.500000,1.500000,1.000000
max,1293.000000,82.000000,40.900000,28.200000,10.200000,19.800000,73.700000,2.300000,6.500000,100.000000,...,10.200000,100.000000,5.300000,9.600000,13.900000,10.600000,2.500000,3.900000,4.400000,1.000000


In [14]:
y = df['target']
X = df.drop(['target'], axis=1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True)

In [16]:
y_train.value_counts(1)

1.0    0.613006
0.0    0.386994
Name: target, dtype: float64

In [17]:
y_test.value_counts(1)

1.0    0.636816
0.0    0.363184
Name: target, dtype: float64

In [18]:
model = RandomForestClassifier()
param_search = { 
                'n_estimators': [i for i in range(10,200,10)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'max_depth' : [i for i in range(1,50)],
                'min_samples_leaf': [i for i in range(1,30)],
                'max_leaf_nodes' : [i for i in range(1,10)],
                'class_weight' : ['balanced', 'balanced_subsample']
                }

rsearch = RandomizedSearchCV(estimator = model, param_distributions = param_search, scoring = 'r2', cv = 2, random_state = 0)
rsearch.fit(X_train, y_train)
best_score = rsearch.best_score_
best_model = rsearch.best_estimator_
y_true = y_test.values
y_pred = best_model.predict(X_test)


In [19]:
best_model

RandomForestClassifier(class_weight='balanced', max_depth=25, max_leaf_nodes=5,
                       min_samples_leaf=28, n_estimators=190)

In [20]:
print('test')
print(classification_report(y_true, y_pred))
print('train')
print(classification_report(y_train, best_model.predict(X_train)))

test
              precision    recall  f1-score   support

         0.0       0.56      0.68      0.61       146
         1.0       0.79      0.69      0.74       256

    accuracy                           0.69       402
   macro avg       0.67      0.69      0.67       402
weighted avg       0.71      0.69      0.69       402

train
              precision    recall  f1-score   support

         0.0       0.59      0.72      0.65       363
         1.0       0.79      0.68      0.73       575

    accuracy                           0.70       938
   macro avg       0.69      0.70      0.69       938
weighted avg       0.71      0.70      0.70       938



In [21]:
Xsn_train, Xsn_test = X_train.loc[:,'gp':], X_test.loc[:,'gp':]

In [22]:
model1 = RandomForestClassifier()
param_search = { 
                'n_estimators': [i for i in range(10,200,10)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'max_depth' : [i for i in range(1,50)],
                'min_samples_leaf': [i for i in range(1,30)],
                'max_leaf_nodes' : [i for i in range(1,10)],
                'class_weight' : ['balanced', 'balanced_subsample']
                }

rsearch1 = RandomizedSearchCV(estimator = model1, param_distributions = param_search, scoring = 'r2', cv = 2, random_state = 0)
rsearch1.fit(Xsn_train, y_train)
best_score1 = rsearch1.best_score_
best_model1 = rsearch1.best_estimator_
y_true1 = y_test.values
y_pred1 = best_model1.predict(Xsn_test)


In [23]:
best_model1

RandomForestClassifier(class_weight='balanced_subsample', max_depth=42,
                       max_features='log2', max_leaf_nodes=8, n_estimators=90)

In [29]:
print('test')
print(classification_report(y_true1, y_pred1))
print('train')
print(classification_report(y_train, best_model1.predict(Xsn_train)))

test
              precision    recall  f1-score   support

         0.0       0.56      0.67      0.61       146
         1.0       0.79      0.70      0.74       256

    accuracy                           0.69       402
   macro avg       0.68      0.69      0.68       402
weighted avg       0.71      0.69      0.70       402

train
              precision    recall  f1-score   support

         0.0       0.61      0.74      0.67       363
         1.0       0.81      0.70      0.75       575

    accuracy                           0.71       938
   macro avg       0.71      0.72      0.71       938
weighted avg       0.73      0.71      0.72       938



In [36]:
param_search = { 
                'n_estimators' : [i for i in range(80,100,5)],
                'max_features' : ['log2'],
                'max_depth' : [i for i in range(40,44)],
                'min_samples_leaf': [i for i in range(6,10)],
                'max_leaf_nodes' : [i for i in range(5,9)],
                'class_weight' : ['balanced_subsample']
                }

gsearch = GridSearchCV(model1, param_search, scoring = 'r2', cv = 4)
gsearch.fit(Xsn_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_
y_true = y_test.values
y_pred = best_model.predict(Xsn_test)

In [32]:
best_model

RandomForestClassifier(class_weight='balanced_subsample', max_depth=41,
                       max_features='log2', max_leaf_nodes=7,
                       min_samples_leaf=9, n_estimators=80)

In [35]:
print('test')
print(classification_report(y_true, y_pred))
print('train')
print(classification_report(y_train, best_model.predict(Xsn_train)))

test
              precision    recall  f1-score   support

         0.0       0.55      0.69      0.61       146
         1.0       0.79      0.68      0.73       256

    accuracy                           0.68       402
   macro avg       0.67      0.69      0.67       402
weighted avg       0.71      0.68      0.69       402

train
              precision    recall  f1-score   support

         0.0       0.59      0.73      0.65       363
         1.0       0.80      0.68      0.73       575

    accuracy                           0.70       938
   macro avg       0.70      0.71      0.69       938
weighted avg       0.72      0.70      0.70       938



In [ ]:
final_model = best_model1.fit(X.loc[:,'gp':], y)

In [ ]:
final_predict = final_model.predict(X.loc[:,'gp':])

In [ ]:
print(classification_report(y, final_predict))

In [ ]:
confusion_matrix(y, final_predict)

In [ ]:
model2 = XGBClassifier()

param_search2 = {  
                "n_estimators": [i for i in range(10,50,5)],
                "max_depth": [i for i in range(20,50,5)],
                "learning_rate": [i for i in np.linspace(0.0001,0.9,10)],
                "subsample": [i for i in np.linspace(0.0001,0.9,10)],
                "gamma": [i for i in np.linspace(0.0001,0.9,10)],
                'objective': ['binary:logistic'],
                "min_child_weight": [i for i in range(20,60,5)],
                }

rsearch2 = RandomizedSearchCV(model2, param_search2, cv=5, n_jobs=1, n_iter=100) 
rsearch2.fit(Xsn_train, y_train)
best_score2 = rsearch2.best_score_
best_model2 = rsearch2.best_estimator_
y_true2 = y_test.values
y_pred2 = best_model2.predict(Xsn_test)

In [ ]:
best_model2

In [ ]:
print('test')
print(classification_report(y_true2, y_pred2))
print('train')
print(classification_report(y_train, best_model2.predict(Xsn_train)))